# Introduction

In [ ]:
import numpy as np
from tvsclib.canonical_form import CanonicalForm
from tvsclib.mixed_system import MixedSystem
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD
import tvsclib.utils as utils

import timeit
import matplotlib.pyplot as plt

This notebook describes the basic concepts of the Time Varying Systems library.

Time varying systems are systems that can be epressed with the equations
$$x_{k+1} = A_k x_k + B_k u_k $$
$$y_k = C_k x_k + D_k u_k .$$
Unlike timeinvariant systems the matricies $A_k$,$B_k$,$C_k$ and $D_k$ can change for the different timeindices $k$.
The combination of $A_k$,$B_k$,$C_k$ and $D_k$ for a certian $k$ is called a stage.

The $x_k$ are the states of the system.

These systems can be represented with a matrix if the input vectors $u_k$ are staked to a single input vector $u$ and the output vectors $y_k$ are analogously stacked to $y$.
Then the input-output behavior of the system can be described with the equation 
$$y = Tu$$

It is also possible to have anticausal systems. These are defiend by:
$$x_{k-1} = A_k x_k + B_k u_k $$
$$y_k = C_k x_k + D_k u_k .$$
Here the computation runs back in time.

The causal part corresponds to the lower triangular part of $T$ wheras the anticausal part corresponds to the upper triangular part of $T$.

<img src="Diagrams/Example_Systems.svg" height="500">

## Create a system

First we will create a system to explore the capabilities of the library.
For this we use the `SystemIdentificationSVD`
The output and input dims have to be given.

The library also allows to print the system.
This gives basic informations about the structure.
It is also possible to get an graphic representation of the system.

In [ ]:
dims_in =  [15, 5,10,10,15]
dims_out = [10,10,10,10,15]
matrix = np.random.rand(sum(dims_out), sum(dims_in))
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)


print(system)
utils.show_system(system)

## Extract causal and anticausal part

As we can see the (mixed)system contains a casual system and an anticausal system.
These can be extracted form the mixed system.

In [ ]:
system_causal = system.causal_system
system_anticausal = system.anticausal_system

## Properties of the system

We now have a look at the properites of the system:

The input and output dims determine a segmentation of the matrix. These are the same for the casual and anticausal part. Therefore these are also defined for a mixed system:

In [ ]:
print("dims_in: ",system.dims_in)
print("dims_out:",system.dims_in)

The state dims are usually different for the causal and anticausal part:

In [ ]:
print("Causal:    ",system.causal_system.dims_state)
print("Anticausal:",system.anticausal_system.dims_state)

Note here that the `len(dims_state)=len(dims_in)+1`. This is because we also have the final state output.

For the casual system `dims_state[k]` is the dim of state `k`.
For anticausal systems the last state is `k=-1`. This state is added at the beginning of the vector. Therefore one has to use `dims_state[k+1]` to get the dim of state `k`.

## Calculate the output of the system

Now we can apply the syste to a input vector $u$.
This can be done using the funtion `compute`.

This returns the output vector $y$ and a vector $x$ containing the states

In [ ]:
#create a input vector
u = np.random.rand(sum(dims_in),1)


x_s, y_s = system.compute(u)

#calcaulate reference
y = matrix@u

print(np.allclose(y_s,y))

## Get properties

It is also possible to get the properties of the system.
These depaned on the observability and reachability matrices.
We can determine if the function is minimal using the functions `is_minimal`
The functions `is_observable` and `is_reachable` are also available.
These can be given an optional tolerance for the determination of the rank.

In [ ]:
system.is_minimal()

This is also possible for the causal and anticausal systems:

In [ ]:
system_causal.is_minimal()

With the functions `ìs_input_normal`,`ìs_input_normal` and `ìs_balanced` we can check if a strict system has a normal form

In [ ]:
system_causal.is_input_normal()

In [ ]:
system_anticausal.is_balanced(tolerance=1e-12)

## Convert system back to matrix

It is also possible to convert the system back to the transfer matrix $T$.
This can be done with the function `to_matrix`.
The function usually uses the block matrix representation

$$T = 
\begin{bmatrix}
   D_1 &       &     &  \cdots   \\
   C_2B_1 & D_2&     &  \cdots   \\
   C_3A_2B_1 & C_3B_2& D_3 & \cdots \\
   \vdots    & \vdots& \vdots & \ddots
\end{bmatrix}
$$
This formualtion is usefull for large matrices.
With the option `use_formula=True` the matrix is calculated using the formula
$$T = D + C(I − ZA)^{−1}ZB$$
The matricies $A$, $B$, $C$ and $D$ consist of the combined $A_i$, $B_i$, $C_i$ and $D_i$ respectiveley. The matrix $Z$ is the shift operator.
This is mainly interesting for theoretical purposes.

The same can be done for the anticausal system.
If `to_matrix` is called on a mixed system the matrix of both the causal and the anticausal are calcaulated and added.

In [ ]:
matrix_rec = system_causal.to_matrix(use_formula=True)+ system_anticausal.to_matrix(use_formula=True)
np.allclose(matrix,matrix_rec)

In [ ]:
matrix_constr = system_causal.to_matrix(use_formula=False)+ system_anticausal.to_matrix(use_formula=False)
np.allclose(matrix,matrix_constr)

Compare the speed of the different implementations

In [ ]:
timeit.timeit(lambda:system_causal.to_matrix(use_formula=False), number=10)

In [ ]:
timeit.timeit(lambda:system_causal.to_matrix(use_formula=True), number=10)

## Semiseperable Matices

Here we want to create a system representing a semiseperable matrix.
Semiseperable matrices have the property that the lower triangular matrix and their upper trinagular matrix have both rank 1.
The inverses of Semiseperable matrices are tridiagonal matrices.
This time we weil create a tridiagonal matrix. After this we will invert the matrix and get an semiseperable matrix. 

This matrix has the property that all submatrices taken form the lower and upper triangular parts have rank $1$.
This also means that the a state dimension of $1$ is sufficent.

In [ ]:
dims_in =  [1]*8
dims_out = [1]*8
matrix = np.random.rand(sum(dims_out), sum(dims_in))
matrix = matrix - np.tril(matrix,-2) - np.triu(matrix,2) # Banded shape
plt.figure()
plt.matshow(abs(matrix))
plt.title("Tridiagonal Matrix")


matrix = np.linalg.inv(matrix)                           # Obscure structure
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T)

plt.figure()
plt.matshow(abs(matrix))
plt.title("Semiseperable Matrix")


system = MixedSystem(S)
system_causal = system.causal_system
system_anticausal = system.anticausal_system

print(f"Causal state dimensions: {system_causal.dims_state}")
print(f"Anticausal state dimensions: {system_anticausal.dims_state}")
